<a href="https://colab.research.google.com/github/katoki001/emo_study/blob/main/Copy_of_Untitled34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-telegram-bot transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.3/737.3 kB 20.5 MB/s eta 0:00:00


In [ ]:
!pip install -q sentence-transformers pandas torch

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from datasets import Dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving emotion_data.csv to emotion_data.csv


In [ ]:
df = pd.read_csv("emotion_data.csv")
df = df.dropna(subset=["statement", "status"])
df = df.sample(n=2000, random_state=42)



label_map = {"Normal":0, "Anxiety":1, "Bipolar":2, "Depression":3, "Suicidal":4}
df['label_id'] = df['status'].map(label_map)
df = df.dropna(subset=['label_id'])
df = df.drop(columns=['Unnamed: 0'])
df['label_id'] = df['label_id'].astype(int)
print(df.isna().sum())

print(df.head())
len(df)


statement    0
status       0
label_id     0
dtype: int64
                                               statement      status  label_id
3008                I'm lazy to complain about it ba ihh      Normal         0
44705  i think the wifi on my iphone is broken it wil...      Normal         0
50186  Good tracking apps? I've been trying to find a...     Bipolar         2
13755  I have recently looked into reddit and found t...  Depression         3
32298                  that's your favorite thing to do?      Normal         0


1852

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer


num_labels = len(label_map)

model_name = "distilbert-base-uncased"

# Define the device for model training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
).to(device)

Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_layer_norm.weight | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_transform.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
classifier.weight       | MISSING    | 
pre_classifier.bias     | MISSING    | 
pre_classifier.weight   | MISSING    | 
classifier.bias         | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(texts, truncation=True, padding=True)
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

dataset = MyDataset(df["statement"].tolist(), df["label_id"].tolist())
loader = DataLoader(dataset, batch_size=16, shuffle=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn as nn



optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

for epoch in range(3):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = {k: v.to(device) for k, v in batch.items()}

        # Remove 'token_type_ids' if present, as DistilBERT models often don't use them
        if 'token_type_ids' in batch:
            del batch['token_type_ids']

        optimizer.zero_grad()

        outputs = model(**batch)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(loader):.4f}")

Epoch 1, Loss: 1.0822
Epoch 2, Loss: 0.6461
Epoch 3, Loss: 0.4352


In [ ]:
model.save_pretrained("my_model")
tokenizer.save_pretrained("my_model")


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('my_model/tokenizer_config.json', 'my_model/tokenizer.json')

In [ ]:
!pip install transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.3 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification # Added AutoModelForSequenceClassification

MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

# Renamed Phi-3 tokenizer and model to avoid conflict with classifier
phi3_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
phi3_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype="auto"
)

# Load the fine-tuned classification model and its tokenizer
# 'device' variable is available from previous cells.
classifier_tokenizer = AutoTokenizer.from_pretrained("my_model")
classifier_model = AutoModelForSequenceClassification.from_pretrained("my_model").to(device)

# Define id2label mapping (label_map is available from previous cells)
id2label = {v: k for k, v in label_map.items()}

# ===============================
# System Prompt (VERY IMPORTANT)
# ===============================
SYSTEM_PROMPT = """
You are a calm, empathetic mental-health support chatbot.
You ask ONLY ONE short, gentle follow-up question per message.
You do NOT repeat the user's words.
You sound natural, human, and supportive.
You never diagnose or give medical advice.
"""


# ===============================
# Chat function (uses phi3_tokenizer and phi3_model)
# ===============================
def phi3_chat(history):
    messages = [{"role": "system", "content": SYSTEM_PROMPT}] + history

    encoded_inputs = phi3_tokenizer.apply_chat_template(
        messages,
        return_tensors="pt",
        add_generation_prompt=True
    ).to(phi3_model.device)

    output = phi3_model.generate(
        input_ids=encoded_inputs['input_ids'],
        attention_mask=encoded_inputs['attention_mask'],
        max_new_tokens=100,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )

    reply = phi3_tokenizer.decode(
        output[0][encoded_inputs['input_ids'].shape[-1]:],
        skip_special_tokens=True
    )

    return reply.strip()

# ===============================
# Classifier function
# ===============================
def detect_state_probs(text):
    classifier_model.eval() # Set model to evaluation mode
    inputs = classifier_tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(device)

    # Remove 'token_type_ids' if present, as DistilBERT models often don't use them
    if 'token_type_ids' in inputs:
        del inputs['token_type_ids']

    with torch.no_grad():
        outputs = classifier_model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)[0]
    return probabilities


# ===============================
# MAIN CHAT LOOP
# ===============================
MAX_TURNS = 5
turn = 0

history = []
all_probs = []

print("🟢 Chat started (type 'exit' to stop)\n")

while turn < MAX_TURNS:
    user_input = input("You: ")

    if user_input.lower() == "exit":
        break

    history.append({"role": "user", "content": user_input})

    # --- Classifier (silent) ---
    probs = detect_state_probs(user_input)
    all_probs.append(probs)

    # --- Bot reply ---
    bot_reply = phi3_chat(history)
    print("Bot:", bot_reply)

    history.append({"role": "assistant", "content": bot_reply})

    turn += 1

if all_probs: # Ensure all_probs is not empty before stacking
    final_probs = torch.stack(all_probs).mean(dim=0)

    print("\n🧠 FINAL MENTAL STATE ESTIMATION")
    print("================================")
    for i, p in enumerate(final_probs):
        print(f"{id2label[i]}: {p.item()*100:.2f}%")
    print("================================")
else:
    print("\nNo chat turns completed, no final mental state estimation.")

Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/104 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 90.00 MiB. GPU 0 has a total capacity of 14.56 GiB of which 61.81 MiB is free. Including non-PyTorch memory, this process has 14.50 GiB memory in use. Of the allocated memory 14.03 GiB is allocated by PyTorch, and 346.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from telegram import Update
from telegram.ext import ApplicationBuilder, MessageHandler, filters, ContextTypes

BOT_TOKEN = "8494603893:AAF3ROgwTIyYSIT3vf9fJLRT5yqK4AMnxY8"

user_states = {}  # chat_id → history & probabilities
MAX_TURNS = 5


In [ ]:
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    chat_id = update.effective_chat.id
    text = update.message.text

    if chat_id not in user_states:
        user_states[chat_id] = {
            "history": [],
            "probs": [],
            "turns": 0
        }

    data = user_states[chat_id]

    data["history"].append({"role": "user", "content": text})

    # --- classifier ---
    probs = detect_state_probs(text)
    data["probs"].append(probs)

    # --- chat response ---
    reply = phi3_chat(data["history"])
    data["history"].append({"role": "assistant", "content": reply})

    await update.message.reply_text(reply)

    data["turns"] += 1

    # --- finish conversation ---
    if data["turns"] >= MAX_TURNS:
        final_probs = torch.stack(data["probs"]).mean(dim=0)

        result = "\n🧠 Final mental state estimation:\n"
        for i, p in enumerate(final_probs):
            result += f"{id2label[i]}: {p.item()*100:.2f}%\n"

        await update.message.reply_text(result)
        del user_states[chat_id]


In [ ]:
!pip install nest_asyncio
import nest_asyncio

nest_asyncio.apply()

app = ApplicationBuilder().token(BOT_TOKEN).build()
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

print("Bot is running...")
app.run_polling()